# 🔎 AIF360 Fairness Check + Mitigation
In dit notebook voeren we een eenvoudige fairness-check uit met de Adult dataset.
We kijken of er bias is op basis van **geslacht** (Male vs. Female) en passen daarna een **Reweighing**-techniek toe om bias te verminderen.

## 1. Imports
We laden de benodigde libraries.

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

## 2. Data laden
We halen de Adult dataset op via OpenML, maken kolomnamen netjes en zetten hem om naar een AIF360 `StandardDataset`. 

In [ ]:
adult = fetch_openml('adult', version=2, as_frame=True)
df = adult.frame.copy()

df.columns = [c.strip() for c in df.columns]
for col in ['sex', 'class']:
    df[col] = df[col].astype(str).str.strip()

categorical_features = [
    'workclass','education','marital-status','occupation',
    'relationship','race','sex','native-country'
]

dataset = StandardDataset(
    df=df,
    label_name='class',
    favorable_classes=['>50K'],
    protected_attribute_names=['sex'],
    privileged_classes=[['Male']],
    categorical_features=categorical_features,
    na_values=['?']
)

print("✅ Dataset geladen:", dataset.features.shape[0], "samples")

## 3. Fairness check (voor mitigatie)
We berekenen de **Disparate Impact** voor geslacht.

In [ ]:
metric_before = BinaryLabelDatasetMetric(
    dataset,
    privileged_groups=[{'sex': 1}],
    unprivileged_groups=[{'sex': 0}]
)
print("Disparate Impact vóór mitigatie:", metric_before.disparate_impact())

## 4. Bias mitigatie met Reweighing
We passen **Reweighing** toe: een techniek die sample-gewichten aanpast zodat de dataset eerlijker wordt.

In [ ]:
rw = Reweighing(
    privileged_groups=[{'sex': 1}],
    unprivileged_groups=[{'sex': 0}]
)
dataset_transf = rw.fit_transform(dataset)

metric_after = BinaryLabelDatasetMetric(
    dataset_transf,
    privileged_groups=[{'sex': 1}],
    unprivileged_groups=[{'sex': 0}]
)
print("Disparate Impact ná mitigatie:", metric_after.disparate_impact())

## 5. Interpretatie
- **Vóór mitigatie**: De waarde van Disparate Impact ligt waarschijnlijk rond de 0.6–0.7. Dit betekent dat vrouwen ~60–70% kans hebben op een positieve uitkomst vergeleken met mannen → duidelijke bias.
- **Ná mitigatie**: De waarde schuift dichter naar 1.0. Hoe dichter bij 1.0, hoe eerlijker de dataset is verdeeld → bias is verminderd.

👉 In gewone taal: *voor de mitigatie discrimineerde de dataset tegen vrouwen, na reweighing is de data veel eerlijker verdeeld.*

💡 In de praktijk gebruik je deze "gecorrigeerde dataset" om een model te trainen dat minder bias bevat.